# QCEW 2022 — County × NAICS **Sector** prep (step-through, with diagnostics)
Filters the annual singlefile to **own_code = '5' (private)**, derives NAICS2 sectors from detailed codes,
and aggregates to county × NAICS2 for benchmarking.


In [1]:
import pandas as pd
from pathlib import Path
import numpy as np

# 1. Point to your downloaded QCEW 2022 annual single file
#    (Rename this to match your actual filename)
INPUT_CSV = "/Users/michaelwalker/RDM_Datalab/rdm-datalab-pipelines/data_raw/qcew/2022.annual.singlefile.csv"

# 2. Load with dtype=str to avoid weird type issues on codes
df = pd.read_csv(INPUT_CSV, dtype=str)

print("Columns:", df.columns.tolist())
print("Shape before filtering:", df.shape)

# ---- 3. Basic type cleanup ----
# These are the standard QCEW column names for the annual single files.
# If your file uses slightly different names, tweak here.
df["year"] = df["year"].astype(str)
df["own_code"] = df["own_code"].astype(str)
df["agglvl_code"] = df["agglvl_code"].astype(str)
df["industry_code"] = df["industry_code"].astype(str)
df["area_fips"] = df["area_fips"].astype(str)

# Numeric fields
for col in ["annual_avg_emplvl", "total_annual_wages"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    else:
        raise ValueError(f"Missing expected column: {col}")

# ---- 4. Filter to 2022, private ownership ----
df = df[df["year"] == "2022"]
df = df[df["own_code"] == "5"]    # private only

# We *don’t* rely on agglvl_code=73 existing; we aggregate
# detailed NAICS up to NAICS2 ourselves.
print("Shape after year/own filter:", df.shape)

# ---- 5. Derive NAICS2 sector code from detailed industry_code ----

def derive_naics2(code: str) -> str | None:
    """
    Derive NAICS2 sector from detailed NAICS code.
    Returns a string like '11', '21', '31-33', '44-45', '48-49', or None
    if it can't be mapped.
    """
    if not isinstance(code, str):
        return None
    code = code.strip()
    # Drop non-NAICS stuff (like '10' total, '90' government, or blanks)
    if not code or not code[0].isdigit():
        return None
    # Some files have 2-digit NAICS already
    if len(code) == 2 and code.isdigit():
        base = code
    else:
        # Use first two digits as base sector
        base = code[:2]
        if not base.isdigit():
            return None

    if base in {"31", "32", "33"}:
        return "31-33"
    if base in {"44", "45"}:
        return "44-45"
    if base in {"48", "49"}:
        return "48-49"

    # Normal 2-digit sectors
    if base in {
        "11","21","22","23","42","51","52","53","54",
        "55","56","61","62","71","72","81","92"
    }:
        return base

    # Otherwise, not a sector we care about
    return None

df["naics2_sector_cd"] = df["industry_code"].apply(derive_naics2)

# Drop rows that don't map cleanly to a NAICS2 sector
before = len(df)
df = df[~df["naics2_sector_cd"].isna()].copy()
after = len(df)
print(f"Dropped {before - after} rows that did not map to NAICS2 sector")
print("Sample naics2_sector_cd:", df["naics2_sector_cd"].unique())

# ---- 6. Aggregate to county × NAICS2 (private) ----

# Keep only 5-digit county FIPS (skip state/US totals)
df["area_fips"] = df["area_fips"].str.zfill(5)
df = df[df["area_fips"].str.len() == 5].copy()

group_cols = ["area_fips", "naics2_sector_cd"]

agg = df.groupby(group_cols, as_index=False).agg(
    qcew_ann_avg_emp_lvl_num=("annual_avg_emplvl", "sum"),
    qcew_ttl_ann_wage_usd_amt=("total_annual_wages", "sum"),
)

# Recompute avg weekly wage from aggregates
# BLS avg annual pay = total_annual_wages / annual_avg_emplvl
# avg weekly wage ~= avg annual pay / 52
agg["qcew_avg_wkly_wage_usd_amt"] = (
    agg["qcew_ttl_ann_wage_usd_amt"] / agg["qcew_ann_avg_emp_lvl_num"] / 52
).round(2)

# Replace inf, -inf with NaN first
agg["qcew_avg_wkly_wage_usd_amt"].replace([np.inf, -np.inf], np.nan, inplace=True)

# Add year and ownership metadata
agg["year_num"] = 2022
agg["own_code"] = "5"  # private

# Rename to match your RDM naming if you like
agg = agg.rename(
    columns={
        "area_fips": "state_cnty_fips_cd",
    }
)

print("Final aggregated shape:", agg.shape)
print(agg.head())
out = agg[["year_num","naics2_sector_cd","state_cnty_fips_cd","own_code", "qcew_ann_avg_emp_lvl_num","qcew_ttl_ann_wage_usd_amt","qcew_avg_wkly_wage_usd_amt"]]
# ---- 7. Save for your pipeline ----

OUTPUT_CSV = "/Users/michaelwalker/RDM_Datalab/rdm-datalab-pipelines/data_clean/qcew/econ_bnchmrk_qcew.csv"
out.to_csv(OUTPUT_CSV, index=False)
print(f"Saved private-only NAICS2 QCEW to data_clean/qcew/econ_bnchmrk_qcew.csv")

Columns: ['area_fips', 'own_code', 'industry_code', 'agglvl_code', 'size_code', 'year', 'qtr', 'disclosure_code', 'annual_avg_estabs', 'annual_avg_emplvl', 'total_annual_wages', 'taxable_annual_wages', 'annual_contributions', 'annual_avg_wkly_wage', 'avg_annual_pay', 'lq_disclosure_code', 'lq_annual_avg_estabs', 'lq_annual_avg_emplvl', 'lq_total_annual_wages', 'lq_taxable_annual_wages', 'lq_annual_contributions', 'lq_annual_avg_wkly_wage', 'lq_avg_annual_pay', 'oty_disclosure_code', 'oty_annual_avg_estabs_chg', 'oty_annual_avg_estabs_pct_chg', 'oty_annual_avg_emplvl_chg', 'oty_annual_avg_emplvl_pct_chg', 'oty_total_annual_wages_chg', 'oty_total_annual_wages_pct_chg', 'oty_taxable_annual_wages_chg', 'oty_taxable_annual_wages_pct_chg', 'oty_annual_contributions_chg', 'oty_annual_contributions_pct_chg', 'oty_annual_avg_wkly_wage_chg', 'oty_annual_avg_wkly_wage_pct_chg', 'oty_avg_annual_pay_chg', 'oty_avg_annual_pay_pct_chg']
Shape before filtering: (3619437, 38)
Shape after year/own filte

/var/folders/yq/thx3vlf97yq5hz7fwld1dmk00000gn/T/ipykernel_58416/3124045973.py:109: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  agg["qcew_avg_wkly_wage_usd_amt"].replace([np.inf, -np.inf], np.nan, inplace=True)


In [25]:
import pandas as pd
import numpy as np

# Parameters — edit as needed
QCEW_RAW = "/Users/michaelwalker/RDM_Datalab/rdm-datalab-pipelines/data_raw/qcew/qcew_annual_raw_2022.csv"   # your QCEW annual CSV (singlefile or concatenated "by area")
YEAR = 2022
OUT = "/Users/michaelwalker/RDM_Datalab/rdm-datalab-pipelines/data_clean/qcew/econ_bnchmrk_qcew.csv"


In [26]:
VALID_SECTORS = {
    "11","21","22","23","31-33","42","44-45","48-49","51","52","53","54",
    "55","56","61","62","71","72","81","92"
}

def normalize_qcew_columns(df):
    if df.columns.duplicated().any():
        df = df.loc[:, ~df.columns.duplicated()].copy()
    lower = {c.lower(): c for c in df.columns}
    def pick(*opts):
        for o in opts:
            if o in lower:
                return lower[o]
        return None
    area     = pick("area_fips","area","fips")
    ind      = pick("industry_code","naics","industry")
    year_col = pick("year")
    aemp     = pick("annual_avg_emplvl","annual_avg_employment","annualaverageemployment","annual_avg_emplv")
    twages   = pick("total_annual_wages","totalannualwages","annual_total_wages","tot_annual_wages")
    awage    = pick("avg_wkly_wage","avg_weekly_wage","average_weekly_wage","annual_avg_wkly_wage")
    agglvl   = pick("agglvl_code","agglevel_code","aggregation_level")
    own      = pick("own_code","ownership","own")
    qtr      = pick("qtr","quarter")
   # need = [area, ind, year_col, aemp, twages, awage, agglvl]
  #  if any(x is None for x in need):
  #      missing = [n for n,x in zip(
  #          ["area_fips","industry_code","year","annual_avg_emplvl","total_annual_wages","avg_weekly_wage","agglvl_code"],
  #          need) if x is None]
  #      raise ValueError(f"Missing required columns (or synonyms): {missing}")
    df = df.rename(columns={
        area: "state_cnty_fips_cd",
        ind: "indstr_cd",
        year_col: "year_num",
        aemp: "qcew_ann_avg_emp_lvl_num",
        twages: "qcew_ttl_ann_wage_usd_amt",
        awage: "qcew_avg_wkly_wage_usd_amt",
        agglvl: "agg_lvl_cd"
    })
  #  if own:
  #      df = df.rename(columns={own: "own_code"})
  #  if qtr:
  #      df = df.rename(columns={qtr: "qtr"})
    return df

In [76]:
def prep_sector(qdf, year=None):
    df = qdf.copy()

    if year is not None and "year_num" in df.columns:
        df = df[df["year_num"].astype(str) == str(year)]

    if "qtr" in df.columns:
        df = df[df["qtr"].astype(str).str.upper().eq("A")]

    if "own_code" in df.columns:
        df["own_code"] = df["own_code"].astype(str).str.strip()
        df = df[df["own_code"] == "5"]
    else:
        df["own_code"] = "5"

    df["state_cnty_fips_cd"] = df["state_cnty_fips_cd"].astype(str).str.zfill(5)
    df = df[df["state_cnty_fips_cd"].str.len() == 5].copy()
        
    df["indstr_cd"] = df["indstr_cd"].astype(str).str.strip()
    df["naics2_sector_cd"] = df["indstr_cd"].apply(derive_naics2)
    df = df[df["naics2_sector_cd"].notna()].copy()

    for c in ["qcew_ann_avg_emp_lvl_num","qcew_ttl_ann_wage_usd_amt","qcew_avg_wkly_wage_usd_amt"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    if "year_num" in df.columns:
        df["year_num"] = pd.to_numeric(df["year_num"], errors="coerce")

    out = (df.groupby(["state_cnty_fips_cd","naics2_sector_cd","year_num","own_code"], as_index=False)
             .agg({
                 "qcew_ann_avg_emp_lvl_num":"sum",
                 "qcew_ttl_ann_wage_usd_amt":"sum"
             }))

    out["qcew_avg_wkly_wage_usd_amt"] = np.where(
        out["qcew_ann_avg_emp_lvl_num"] > 0,
        out["qcew_ttl_ann_wage_usd_amt"] / (out["qcew_ann_avg_emp_lvl_num"] * 52.0),
        np.nan
    )
    out["qcew_avg_wkly_wage_usd_amt"] = out["qcew_avg_wkly_wage_usd_amt"].replace([np.inf,-np.inf], np.nan).round(2)
    out["own_cd"] = out["own_code"]
    out["state_fips_cd"] = out["state_cnty_fips_cd"].str[:2]
    out["cnty_fips_cd"] = out["state_cnty_fips_cd"].str[2:]

    assert out.duplicated(subset=["state_cnty_fips_cd","naics2_sector_cd","year_num","own_cd"]).sum() == 0
    for c in ["qcew_ann_avg_emp_lvl_num","qcew_ttl_ann_wage_usd_amt"]:
        assert (out[c].dropna() >= 0).all()

    return out[["year_num","naics2_sector_cd","state_cnty_fips_cd","own_cd",
                "qcew_ann_avg_emp_lvl_num","qcew_ttl_ann_wage_usd_amt","qcew_avg_wkly_wage_usd_amt"]]


In [77]:

raw = pd.read_csv(QCEW_RAW, dtype=str)
#print("Columns:", list(raw.columns)[:15], "...")
#print("Years:", sorted(raw["year"].unique().tolist()) if "year" in raw.columns else "n/a")
#print("Agglvl codes (top 10):")
#print(raw["agglvl_code"].value_counts().head(10) if "agglvl_code" in raw.columns else "n/a")
#print("Own codes:", sorted(raw["own_code"].unique().tolist()) if "own_code" in raw.columns else "n/a")
#if "qtr" in raw.columns:
#    print("QTR values:", sorted(raw["qtr"].unique().tolist()))
raw = normalize_qcew_columns(raw)
mask = raw["agg_lvl_cd"] == "73"
raw[mask]


,state_cnty_fips_cd,own_code,indstr_cd,agg_lvl_cd,size_code,year_num,qtr,disclosure_code,annual_avg_estabs,qcew_ann_avg_emp_lvl_num,...,oty_total_annual_wages_chg,oty_total_annual_wages_pct_chg,oty_taxable_annual_wages_chg,oty_taxable_annual_wages_pct_chg,oty_annual_contributions_chg,oty_annual_contributions_pct_chg,oty_annual_avg_wkly_wage_chg,oty_annual_avg_wkly_wage_pct_chg,oty_avg_annual_pay_chg,oty_avg_annual_pay_pct_chg
2840,01001,1,1021,73,0,2022,A,NaN,4,74,...,391324,9.4,0,0.0,0,0.0,5,0.4,291,0.5
2841,01001,1,1028,73,0,2022,A,NaN,6,13,...,170325,20.7,0,0.0,0,0.0,170,12.8,8840,12.8
2868,01001,2,1021,73,0,2022,A,NaN,2,7,...,0,0,0,0,0,0,0,0,0,0
2869,01001,2,1025,73,0,2022,A,NaN,4,388,...,0,0,0,0,0,0,0,0,0,0
2870,01001,2,1026,73,0,2022,A,NaN,1,5,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3053002,78030,5,1024,73,0,2022,A,NaN,389,1649,...,4946750,6.1,-1703918,-3.8,-35885,-3.4,58,6.2,3019,6.2
3053003,78030,5,1025,73,0,2022,A,NaN,160,1128,...,3094092,6.3,1199829,5.0,15673,2.8,21,2.4,1070,2.4
3053004,78030,5,1026,73,0,2022,A,NaN,243,3492,...,17138453,16.3,5898386,7.2,-103031,-4.6,69,11.4,3591,11.5
3053005,78030,5,1027,73,0,2022,A,NaN,164,413,...,0,0,0,0,0,0,0,0,0,0


In [78]:

out = prep_sector(raw, year=YEAR)
print("Rows:", len(out))
out.head(10)


#1 69900
{'3', '5', '1', '2'}
#2 9344
#3 9344
#4 0
[]
Rows: 0


/var/folders/yq/thx3vlf97yq5hz7fwld1dmk00000gn/T/ipykernel_38461/422445979.py:52: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  agg["qcew_avg_wkly_wage_usd_amt"].replace([np.inf, -np.inf], np.nan, inplace=True)


,year_num,naics2_sector_cd,state_cnty_fips_cd,state_fips_cd,cnty_fips_cd,own_cd,qcew_ann_avg_emp_lvl_num,qcew_ttl_ann_wage_usd_amt,qcew_avg_wkly_wage_usd_amt


In [7]:
out["own_cd"].value_counts

<bound method IndexOpsMixin.value_counts of 0        5
1        5
2        5
3        5
4        5
        ..
59126    5
59127    5
59128    5
59129    5
59130    5
Name: own_cd, Length: 59131, dtype: object>

In [21]:

out.to_csv(OUT, index=False)
print("Wrote:", OUT)


Wrote: /Users/michaelwalker/RDM_Datalab/rdm-datalab-pipelines/data_clean/qcew/econ_bnchmrk_qcew.csv


In [14]:
import pandas as pd
import numpy as np

df = pd.read_csv("qcew_county_naics_sector_2022.csv")

In [15]:
df.head(10)

,state_fips,county_fips,naics_sector,year,annual_avg_emplvl,total_annual_wages,avg_weekly_wage
0,1,1,11,2022,97,7271752,1441.663759
1,1,1,21,2022,79,5383592,1310.514119
2,1,1,22,2022,84,12421322,2843.709249
3,1,1,23,2022,527,28459329,1038.510035
4,1,1,31-33,2022,1534,104157352,1305.753585
5,1,1,42,2022,418,28889133,1329.091507
6,1,1,44-45,2022,1689,52853052,601.779068
7,1,1,48-49,2022,145,7332786,972.518037
8,1,1,51,2022,35,2896883,1591.693956
9,1,1,52,2022,306,18320054,1151.335721


In [16]:
# 1) Key uniqueness
key = ["state_fips","county_fips","naics_sector","year"]
dups = df.duplicated(subset=key).sum()
print("Duplicate key rows:", dups)
assert dups == 0

Duplicate key rows: 0


In [17]:
# 2) Non-negatives & basic types
for c in ["annual_avg_emplvl","total_annual_wages","avg_weekly_wage"]:
    assert (df[c].dropna() >= 0).all(), f"Negative values in {c}"

In [18]:
len(df)

59131

In [19]:
# 3) Avg weekly wage recompute check (should match within tiny tolerance)
recomp = (df["total_annual_wages"] / (df["annual_avg_emplvl"] * 52)).round(2)
mismatch = (df["avg_weekly_wage"].round(2) - recomp).abs() > 0.01
print("Avg weekly wage mismatches:", mismatch.sum())

Avg weekly wage mismatches: 0


In [20]:
# 4) Sector coverage (expect the NAICS sector set incl. 31-33, 44-45, 48-49)
print("Sectors present:", sorted(df["naics_sector"].unique().tolist()))

Sectors present: ['11', '21', '22', '23', '31-33', '42', '44-45', '48-49', '51', '52', '53', '54', '55', '56', '61', '62', '71', '72', '81']


In [21]:
# 5) County-year wage shares sum ~ 1.0
shares = df.assign(
    wage_share = df["total_annual_wages"] /
                 df.groupby(["state_fips","county_fips","year"])["total_annual_wages"].transform("sum")
)
share_sums = shares.groupby(["state_fips","county_fips","year"])["wage_share"].sum()
print("Share sums ~1.0? 5-number summary:\n", (share_sums - 1).describe())

Share sums ~1.0? 5-number summary:
 count    3274.000000
mean       -0.001833
std         0.042776
min        -1.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         0.000000
Name: wage_share, dtype: float64


In [22]:
# 6) Outliers in wages per employee (annual)
df["wage_per_emp"] = df["total_annual_wages"] / df["annual_avg_emplvl"]
print("wage_per_emp (annual) summary:\n", df["wage_per_emp"].describe())
suspicious = df[(df["wage_per_emp"] < 10000) | (df["wage_per_emp"] > 300000)]
print("Potential outliers:", len(suspicious))


wage_per_emp (annual) summary:
 count    4.400400e+04
mean     5.331257e+04
std      3.042128e+04
min      2.879792e+03
25%      3.406708e+04
50%      4.864711e+04
75%      6.501049e+04
max      1.395664e+06
Name: wage_per_emp, dtype: float64
Potential outliers: 124


In [23]:
import pandas as pd
import numpy as np

# Always read with explicit dtypes (or coerce then zfill)
df = pd.read_csv(
    "qcew_county_naics_sector_2022.csv",
    dtype=str  # simplest: read all as strings
)

# Normalize FIPS columns to zero-padded strings
df["state_fips"]  = df["state_fips"].astype(str).str.replace(r"\D","", regex=True).str.zfill(2)
df["county_fips"] = df["county_fips"].astype(str).str.replace(r"\D","", regex=True).str.zfill(3)

# (Optional) make sure year is numeric if you need it later
df["year"] = pd.to_numeric(df["year"], errors="coerce")

# Now this should return rows for Los Angeles County, CA (06 / 037)
la = (df[(df["state_fips"]=="06") & (df["county_fips"]=="037")]
        .sort_values("total_annual_wages", ascending=False)
        .head(10))
la



,state_fips,county_fips,naics_sector,year,annual_avg_emplvl,total_annual_wages,avg_weekly_wage
3802,06,037,55,2022,61278,9121719529,2862.653533894709
3800,06,037,53,2022,88372,7750573929,1686.6145230911984
3804,06,037,61,2022,110091,7645003932,1335.4343805089916
3808,06,037,81,2022,148231,7588753710,984.5280092973389
3805,06,037,62,2022,751456,40850880348,1045.4289444041353
3801,06,037,54,2022,308801,38936838635,2424.8151993205483
3798,06,037,51,2022,232524,33151705736,2741.793546538786
3794,06,037,31-33,2022,320994,27986279672,1676.659644728562
3790,06,037,11,2022,4773,253199289,1020.158620606295
3799,06,037,52,2022,124825,20473361275,3154.163717666271


In [24]:
df.dtypes
#df["state_fips"] == '39'

state_fips            object
county_fips           object
naics_sector          object
year                   int64
annual_avg_emplvl     object
total_annual_wages    object
avg_weekly_wage       object
dtype: object

In [25]:
oh_filter = df["state_fips"] == '39'

In [26]:
df[oh_filter]

,state_fips,county_fips,naics_sector,year,annual_avg_emplvl,total_annual_wages,avg_weekly_wage
37528,39,001,11,2022,0,0,NaN
37529,39,001,21,2022,0,0,NaN
37530,39,001,22,2022,56,4599542,1579.5130494505495
37531,39,001,23,2022,287,18878620,1264.983918520504
37532,39,001,31-33,2022,752,52487413,1342.2517645253683
...,...,...,...,...,...,...,...
39200,39,999,61,2022,2975,166855747,1078.5762572721396
39201,39,999,62,2022,6013,453508055,1450.4089057043072
39202,39,999,71,2022,472,28015321,1141.432570078227
39203,39,999,72,2022,1099,59072081,1033.6683873451389
